## Data Preprrocessing
Author: Dalyah Aljamal, April 21, 2018
#This file read data (Training (X,Y) , testing(X,Y))
#Perform MiniBatching
#Image preprocessing: 
1. Segmentation (not complete)
2. Resizing
3. Normalization (not complete)


#Dp preprossing for all data once and Save all the data in a folder (ready_data) (notcomplete)

In [47]:
using Plots
using Knet;
using Compat,GZip 
using Images
using Colors
using Images
using ImageSegmentation


In [48]:
function seg_image(img, xDim, yDim)
    
    println(typeof(img))
    segments = felzenszwalb(img, 300, 100); # k=300 (the merging threshold), min_size = 100 (smallest number of pixels/region)

    seg_img=Any[]
    ymax=0;
    ymin=500000;
    xmax=0;
    xmin=500000;
    
    for i in 1:xDim
        seg_cols=Any[]
        for j in 1:yDim
            hand_i=2# for all training images
            if labels_map(segments)[i,j] == hand_i
            #push!(seg_cols,img[i,j])
                if i > xmax
                    xmax=i;
                end
                if i< min
                    xmin=i;
                end
                if j > xmax
                    ymax=j;
                end
                if j< min
                    ymin=j;
                end
                
            end
        end
            #push!(seg_img,seg_cols)
    end
    #filter(x->x%2==0,[1,2,3,5])
    seg_img= filter(i,j->labels_map(segments)[i,j] == 2, img)
    return seg_img
    #This function works fine, but produces Array{Any,1} img while I need it in 2d format
   
end
   

seg_image (generic function with 1 method)

In [20]:
#Reading Training and Testing img labels

#HOW THE DATA IS ORGANIZED

#Each line is corresponding to one image.
#Each line has 16x3 numbers, which indicates (x, y, z) of 16 joint locations. Note that these are joint CENTRE locations.
#Note that (x, y) are in pixels and z is in mm.
#The order of 16 joints is Palm, Thumb root, Thumb mid, Thumb tip,
#Index root, Index mid, Index tip, Middle root, Middle mid, Middle tip, Ring root,
#Ring mid, Ring tip, Pinky root, Pinky mid, Pinky tip.

function read_labels(label_path)
    labels=Any[]
    open(label_path) do f
         # loop through lines in file (images)
            #loop through all words in a line (joint locations)
                #skip first word
                #stack all joint locations on top of each other
                #return imgjoints vector(48,1) or (49, 1) as first element is the name of img
            #horizantally concatanate imgjoints into labels matrix (48, #lines)
        for ln in eachline(f)
            if(ln=="")
                continue;
            end
            img_joints= Array{Float32}(48)
            word= split(ln)
            for i in 2:49
                w=parse(Float32, word[i])
                img_joints[i-1] = w
            end
            if length(labels) ==0
                labels=img_joints
            else
                labels= hcat(labels, img_joints)
            end
        end
        
    end
    return labels
end



read_labels (generic function with 1 method)

In [ ]:
#Extracting Data Zip Folders
using InfoZIP
using ZipFile
#Since Training file was corupted
#$ zip -FFv Training.zip --out fixed.zip
#InfoZIP.unzip("fixed.zip", "Data/")
#InfoZIP.unzip("Training.zip", "Data/")
#InfoZIP.unzip("Testing.zip", "Data/")

#They are already extracted on my AWS image: HandPosE_Dalyah_v6

In [42]:
## read images function
function read_img(rootpath, arr1,arr2, Atype)
images = Any[]
c=0;
for i =1:length(arr1)
    for j=1:length(arr2)
        
        path2 = string(rootpath,"/",arr1[i],"/",arr2[j])
        x =readdir(path2) #save all img names in arr1[i]/arr2[j] in x var
        for k=1:length(x)
            c=c+1;
            path3 = string(path2,"/",x[k])
            img = load(path3)
            #Transfer to gray scale
            img2=Float64.(Gray.(img))
            #imshow(img)
            #imshow(img2)
            #println(typeof(img))
            #println(size(img2))
            
            #Resize img to (128,128)
            sz=(128,128)
            σ = map((o,n)->0.75*o/n, size(img), sz)
            kern = KernelFactors.gaussian(σ)   # from ImageFiltering
            img3 = imresize(imfilter(img, kern, NA()), sz)
            img4=reshape(img3,(128,128,1,1))
            img5= convert(Array{Float32},img4)
            println(summary(img5))
            #img5= convert(Atype,img5)
            push!(images,img5)
        end
    end
end
    images=cat(4,images...)
    #images2=convert(Atype, images)
    return images
end


read_img (generic function with 2 methods)

In [39]:
function read_tst_img(rootpath, arr1, Atype)
images = Any[]
for i =1:length(arr1)
        path2 = string(rootpath,"/",arr1[i])
        x =readdir(path2) #save all img names in arr1[i]/arr2[j] in x var
        for k=1:length(x)
            path3 = string(path2,"/",x[k])
            img = load(path3)
            
            #Transfer to gray scale
            img2=Float64.(Gray.(img))
            #imshow(img)
            #imshow(img2)
            println(typeof(img))
            println(size(img2))
            #Resize img to (128,128)
            sz=(128,128)
            σ = map((o,n)->0.75*o/n, size(img), sz)
            kern = KernelFactors.gaussian(σ)   # from ImageFiltering
            img3 = imresize(imfilter(img, kern, NA()), sz)
            img4=reshape(img3,(128,128,1,1))
            img5= convert(Array{Float32},img4)
            println(summary(img5))
            #img5= convert(Atype,img5)
            push!(images,img5)
        
    end
end
    images=cat(4,images...)
    #images2=convert(Atype, images)
    return images
end


read_tst_img (generic function with 1 method)

In [44]:
#Mini Batching


function dataset_minibatching(BS, data, labels, Atype)
    all_minibatches=Any[]
    for i in 1:BS:size(data,4)
        b1 = min(i+BS-1, size(data, 4))
        batch=data[:,:,:,i:b1]
        batch=convert(Atype, batch)
        lab= convert(Atype, labels[:,i:b1])
        push!(all_minibatches, (batch, lab))
    end
    
    return  all_minibatches #u can acess single batch by all_minibatches[i]
end



dataset_minibatching (generic function with 1 method)

## Preparing all Data

In [51]:
#TRY IF SegmentationFUN WORKS
#img= load("imgDraft/Training/1/123/image_0000.png")
#xDim=240
#yDim=320
#seg_img=seg_image(img,xDim,yDim);
#println(typeof(seg_img)); #The type of return img should be fixed
#println(size(seg_img,1));
#println(size(seg_img,2));

In [ ]:
#trn_labelpath= "Data/Training/labels.txt"
#tst1_labelpath= "Data/Testing/test_seq_1.txt"
#tst2_labelpath= "Data/Testing/test_seq_2.txt"

#trn_labels= read_labels(trn_labelpath) # (48, )
#tst1_labels= read_labels(tst1_labelpath) # (48, )
#tst2_labels= read_labels(tst2_labelpath) # (48, )

#println(map(summary,(trn_labels, tst1_labels, tst2_labels)))

In [ ]:
#rootpath="imgDraft" # should change based on path on Machine you work on
#arr1 = ["1","2"]
#arr2 = ["123","321"]
#images= read_img(rootpath,arr1,arr2);
#for x in images
  #  println(size(x))
  #  println(typeof(x))
#end
#println(length(images))

In [ ]:
#Minibatch all dataset & save it in ready_data folder
#link for saving
# https://github.com/JuliaIO/JLD.jl